In [64]:
from keras.layers import *
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys

In [2]:
def load(name):
    # Loads the videos and converts the labels into one-hot encoding for Keras
    X = np.load(name + ".npy")
    y = np.load(name + "-labels.npy")
    y_one_hot_encoded = np.zeros([y.shape[0], np.unique(y).size])
    y_one_hot_encoded[range(y.shape[0]), y] = 1
    return X, y_one_hot_encoded

In [3]:
epochs = 10
batch_size = 32
learning_rate = 1e-5

# Loads the preprocessed datasets
print("Dataset loading..", end = " ")
X_train, y_train = load("training")
X_val, y_val = load("validation")
X_test, y_test = load("test")
print("Dataset loaded!")

Dataset loading.. Dataset loaded!


In [4]:
X_train = X_train.transpose(0,1,3,4,2)
X_val = X_val.transpose(0,1,3,4,2)
X_test = X_test.transpose(0,1,3,4,2)

In [60]:
 def c3d():
        model = Sequential()
        # 1st layer group
        model.add(Conv3D(32, (3, 3, 3), activation='relu',
                         name='conv1',
                         input_shape=(45,24,24,3)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(1, 2, 2),
                         name='pool1'))
        # 2nd layer group
        model.add(Conv3D(32, (3, 3, 3), activation='relu',
                         name='conv2'))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                         name='pool2'))
        # 3rd layer group
        model.add(Conv3D(64, (3, 3, 3), activation='relu',
                         name='conv3a'))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                         name='pool3'))
        model.add(Flatten())
        
        # FC layers group
        model.add(Dense(2048, activation='relu', name='fc6'))
        model.add(Dropout(0.5))
        model.add(Dense(1024, activation='relu', name='fc7'))
        model.add(Dropout(0.5))
        model.add(Dense(y_train.shape[1], activation='softmax'))

        return model

In [61]:
model = c3d()

In [62]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr = learning_rate), metrics=["accuracy"])
print(model.summary())

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 43, 22, 22, 32)    2624      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 42, 11, 11, 32)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 40, 9, 9, 32)      27680     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 20, 4, 4, 32)      0         
_________________________________________________________________
conv3a (Conv3D)              (None, 18, 2, 2, 64)      55360     
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 9, 1, 1, 64)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 576)             

In [65]:
# settings for reducing the learning rate
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2, min_lr = 0.0001, verbose = 1)

# Training the model on the training set, with early stopping using the validation set
callbacks = [EarlyStopping(patience = 7), reduce_lr]

In [66]:
model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, shuffle = True, validation_data = (X_val, y_val), callbacks = callbacks)

Epoch 1/10
240/240 [==============================] - 3873s 16s/step - loss: 2.2515 - accuracy: 0.2975 - val_loss: 1.9729 - val_accuracy: 0.4353
Epoch 2/10
240/240 [==============================] - 1203s 5s/step - loss: 1.8385 - accuracy: 0.4438 - val_loss: 1.6365 - val_accuracy: 0.4787
Epoch 3/10
240/240 [==============================] - 1202s 5s/step - loss: 1.6306 - accuracy: 0.4806 - val_loss: 1.5193 - val_accuracy: 0.5240
Epoch 4/10
240/240 [==============================] - 1204s 5s/step - loss: 1.5374 - accuracy: 0.5104 - val_loss: 1.4470 - val_accuracy: 0.5400
Epoch 5/10
240/240 [==============================] - 1188s 5s/step - loss: 1.4761 - accuracy: 0.5270 - val_loss: 1.4009 - val_accuracy: 0.5507
Epoch 6/10
240/240 [==============================] - 1129s 5s/step - loss: 1.4376 - accuracy: 0.5351 - val_loss: 1.3629 - val_accuracy: 0.5553
Epoch 7/10
240/240 [==============================] - 1130s 5s/step - loss: 1.4000 - accuracy: 0.5455 - val_loss: 1.3388 - val_accuracy